In [30]:
# imports
import os

import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, recall_score

In [3]:
# configs
PROJECT_PATH = "/home/alex/paper-2025-anonymous-submission"

In [5]:
# load data

dataset_simple = pd.read_csv(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_predictions/classification_simple.csv"
    )
)

dataset_extended = pd.read_csv(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_predictions/classification_extended.csv"
    )
)

In [7]:
dataset_simple["gigachat_lite_extended"] = dataset_extended["gigachat_lite_extended"]
dataset_simple["gigachat_max_extended"] = dataset_extended["gigachat_max_extended"]
dataset_simple["yagpt4_extended"] = dataset_extended["yagpt4_extended"]
dataset_simple["mistral_extended"] = dataset_extended["mistral_extended"]
dataset_simple["gpt4o_extended"] = dataset_extended["gpt4o_extended"]

In [9]:
dataset_simple = dataset_simple[dataset_simple["is_word_play"]]

In [10]:
dataset_simple

,Unnamed: 0,annotations,summary,is_word_play,date,article_url,headline,lead,gigachat_lite_simple,gigachat_max_simple,yagpt4_simple,mistral_simple,gpt4o_simple,gigachat_lite_extended,gigachat_max_extended,yagpt4_extended,mistral_extended,gpt4o_extended
1,1,"[{'headline_substring': 'стушевалась', 'start_...",Окончание летних каникул и сезона шашлыков при...,True,2021-08-09,https://www.kommersant.ru/doc/4976692,Свинина стушевалась,Началось сезонное снижение цен,False,True,False,False,True,False,True,True,False,True
2,2,"[{'headline_substring': '«Истока»', 'start_ind...","Как стало известно “Ъ”, главное военное следст...",True,2023-09-11,https://www.kommersant.ru/doc/6210688,Исток с «Истока»,Раскрыта преступная группа несунов с оборонног...,True,True,False,False,True,True,True,False,False,True
3,3,"[{'headline_substring': 'Налетай, позеленело',...","Как выяснил “Ъ”, крупнейший инвестор в солнечн...",True,2021-04-08,https://www.kommersant.ru/doc/4928249,"Налетай, позеленело",Компания «Хевел» выставила электроэнергию со с...,False,False,False,False,True,True,True,True,False,True
4,4,[{'headline_substring': 'Гриппы по осени счита...,В России стартовала кампания прививок против г...,True,2023-09-11,https://www.kommersant.ru/doc/6210686,Гриппы по осени считают,В России началась массовая вакцинация от вируса,False,True,False,False,True,True,True,True,False,True
7,7,[{'headline_substring': 'Перестановочная часть...,В среду в двух известных московских театрах од...,True,2021-10-27,https://www.kommersant.ru/doc/5051715,Перестановочная часть,МХТ имени Чехова и МХАТ имени Горького меняют ...,True,True,False,False,True,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,"[{'headline_substring': 'назад в будущее', 'st...",Группа «Аэрофлот» рассчитывает перевезти как м...,True,2023-09-13,https://www.kommersant.ru/doc/6212002,Полет назад в будущее,«Аэрофлот» хочет через год вернуться к уровню ...,True,True,True,False,True,True,True,True,False,True
2496,2496,"[{'headline_substring': 'Сияние', 'start_index...",В прокате — «Корабль призраков» (Haunting of T...,True,2023-08-06,https://www.kommersant.ru/doc/6147302,Сияние кубрика,«Корабль призраков» в российском прокате,True,False,False,False,True,True,True,False,False,True
2497,2497,"[{'headline_substring': 'недоперерабатывают', ...",Российским нефтекомпаниям в августе пока не уд...,True,2023-08-13,https://www.kommersant.ru/doc/6159947,Нефтяники недоперерабатывают,НПЗ в августе пока не увеличили прием нефти,True,True,False,False,True,True,True,False,False,True
2498,2498,"[{'headline_substring': 'Pablo будешь', 'start...","СКР начал проверку клипа «Pablo», снятого рэпе...",True,2021-11-20,https://www.kommersant.ru/doc/5087148,Pablo будешь,СКР проверяет творчество рэпера Моргенштерна н...,True,True,False,False,True,True,True,False,False,True


In [22]:
def extract_annotations_labels(annotations):
    
    annotations = eval(annotations)
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict):
                all_wordplay_type.append(
                    annot["wordplay_type"]
                )
            elif isinstance(annot, list):
                all_wordplay_type.append(
                    annot[0]["wordplay_type"]
                )
    
    return list(set(all_wordplay_type))


In [23]:
dataset_simple["all_wordplay_type"] = dataset_simple["annotations"].apply(extract_annotations_labels)


/tmp/ipykernel_285000/1390876881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_simple["all_wordplay_type"] = dataset_simple["annotations"].apply(extract_annotations_labels)


In [24]:
set(dataset_simple["all_wordplay_type"].explode().tolist())

{'Collocation',
 'Homonymy',
 'Idiom',
 'Nonce word',
 'Oxymoron',
 'Phonetic similarity',
 'Polysemy',
 'Reference',
 nan}

In [36]:
all_types = [
 'Polysemy',
 'Homonymy',
 'Phonetic similarity',
 'Collocation',
 'Idiom',
 'Reference',
 'Nonce word',
 'Oxymoron',
 ]

In [37]:
dataset_simple.columns

Index(['Unnamed: 0', 'annotations', 'summary', 'is_word_play', 'date',
       'article_url', 'headline', 'lead', 'gigachat_lite_simple',
       'gigachat_max_simple', 'yagpt4_simple', 'mistral_simple',
       'gpt4o_simple', 'gigachat_lite_extended', 'gigachat_max_extended',
       'yagpt4_extended', 'mistral_extended', 'gpt4o_extended',
       'all_wordplay_type'],
      dtype='object')

In [41]:
for tt in all_types:
    tmp_df = dataset_simple.copy()
    tmp_df["is_need"] = tmp_df["all_wordplay_type"].apply(lambda x: True if tt in x else False)
    tmp_df = tmp_df[tmp_df["is_need"]]
    

    gc_lite_simple = np.round(recall_score(tmp_df["is_word_play"], tmp_df["gigachat_lite_simple"]), 2)
    gc_lite_extended = np.round(recall_score(tmp_df["is_word_play"], tmp_df["gigachat_lite_extended"]), 2)

    gc_max_simple = np.round(recall_score(tmp_df["is_word_play"], tmp_df["gigachat_max_simple"]), 2)
    gc_max_extended = np.round(recall_score(tmp_df["is_word_play"], tmp_df["gigachat_max_extended"]), 2)

    gc_yagpt_simple = np.round(recall_score(tmp_df["is_word_play"], tmp_df["yagpt4_simple"]), 2)
    gc_yagpt_extended = np.round(recall_score(tmp_df["is_word_play"], tmp_df["yagpt4_extended"]), 2)

    gc_mistral_simple = np.round(recall_score(tmp_df["is_word_play"], tmp_df["mistral_simple"]), 2)
    gc_mistral_extended = np.round(recall_score(tmp_df["is_word_play"], tmp_df["mistral_extended"]), 2)

    gc_gpt4o_simple = np.round(recall_score(tmp_df["is_word_play"], tmp_df["gpt4o_simple"]), 2)
    gc_gpt4o_extended = np.round(recall_score(tmp_df["is_word_play"], tmp_df["gpt4o_extended"]), 2)

    print(
        f"{tt}      & {tmp_df.shape[0]} & {gc_lite_simple} & {gc_lite_extended} & {gc_max_simple} & {gc_max_extended} & {gc_yagpt_simple} & {gc_yagpt_extended} & {gc_mistral_simple} & {gc_mistral_extended} & {gc_gpt4o_simple} & {gc_gpt4o_extended} \\\\"
    )

Polysemy      & 168 & 0.56 & 0.74 & 0.57 & 0.57 & 0.05 & 0.23 & 0.0 & 0.0 & 0.88 & 0.86 \\
Homonymy      & 22 & 0.5 & 0.59 & 0.5 & 0.64 & 0.14 & 0.23 & 0.0 & 0.0 & 0.68 & 0.82 \\
Phonetic similarity      & 88 & 0.4 & 0.74 & 0.44 & 0.58 & 0.1 & 0.15 & 0.0 & 0.0 & 0.81 & 0.9 \\
Collocation      & 393 & 0.47 & 0.7 & 0.48 & 0.58 & 0.09 & 0.2 & 0.0 & 0.0 & 0.78 & 0.87 \\
Idiom      & 164 & 0.5 & 0.74 & 0.49 & 0.65 & 0.12 & 0.38 & 0.0 & 0.0 & 0.87 & 0.96 \\
Reference      & 326 & 0.49 & 0.71 & 0.44 & 0.58 & 0.06 & 0.23 & 0.0 & 0.0 & 0.76 & 0.85 \\
Nonce word      & 166 & 0.52 & 0.81 & 0.45 & 0.6 & 0.21 & 0.27 & 0.0 & 0.0 & 0.87 & 0.96 \\
Oxymoron      & 34 & 0.68 & 0.79 & 0.68 & 0.71 & 0.21 & 0.41 & 0.0 & 0.0 & 0.85 & 0.82 \\
